In [2]:
from deepeval.benchmarks import MMLU
from deepeval.benchmarks.tasks import MMLUTask
import deepeval
import transformers
import torch

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM

class Model(DeepEvalBaseLLM):
    def __init__(
        self,
        model_name
    ):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name)
        self.pipe = transformers.pipeline(
                        "text-generation",
                        model=model_name,
                        torch_dtype=torch.bfloat16,
                        trust_remote_code=True,
                        max_new_tokens=100,
                        do_sample=True,
                        device_map="auto",
                    )

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        # device = "cuda" # the device to load the model onto
#         device = "cuda"

#         model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
#         self.model.to(device)

#         generated_ids = self.model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
#         print(prompt)
#         print(self.tokenizer.batch_decode(generated_ids), "\n")
        # return self.tokenizer.batch_decode(generated_ids)

        return [self.pipe(prompt)[0]['generated_text']]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Mistral 7B"
    
# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_name,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     max_new_tokens=100,
#     do_sample=True,
#     device_map="auto",
# )

In [3]:
benchmark = MMLU(
    tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.SOCIOLOGY],
    n_shots=3
)

In [4]:
from deepeval.benchmarks import TruthfulQA
from deepeval.benchmarks.tasks import TruthfulQATask
from deepeval.benchmarks.modes import TruthfulQAMode

# Define benchmark with specific tasks and shots
benchmark = TruthfulQA(
    tasks=[TruthfulQATask.ADVERTISING, TruthfulQATask.FICTION],
    mode=TruthfulQAMode.MC2
)

In [3]:
falcon_7b = Model(model_name="tiiuae/falcon-7b-instruct")
# print(falcon_7b.generate("Write me a joke"))

Loading checkpoint shards: 100%|██████████| 2/2 [00:30<00:00, 15.27s/it]
2024-05-20 02:49:33.488609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:03<?, ?it/s]

KeyboardInterrupt



In [ ]:
falcon_results = benchmark.evaluate(model=falcon_7b)
print("falcon-7b-instruct - Overall Score: ", falcon_results)

In [ ]:
mistral_7b = Model(model_name="mistralai/Mistral-7B-v0.1")
mistral_7b_results = benchmark.evaluate(model=mistral_7b)
print("mistral_7b - Overall Score: ", mistral_7b_results)

In [ ]:
def calc(model_name):
    model = Model(model_name=model_name)
    model_results = benchmark.evaluate(model=model)
    print(f"{model_name} - Overall Score: ", model_results)
    
    return model_results

In [ ]:
results = []
models_to_check = ["/home/sslashinin/kovakimyan/diploma/trl/examples/sft_mistral_paralccorp/checkpoint-20",
                   "/home/sslashinin/kovakimyan/diploma/trl/examples/sft_mistral_not_toxic/checkpoint-15",
                   "/home/sslashinin/kovakimyan/diploma/trl/examples/sft_falcon_paralccorp/checkpoint-20",
                   "/home/sslashinin/kovakimyan/diploma/trl/examples/sft_falcon_not_toxic/checkpoint-10",
                   
                   "/home/sslashinin/kovakimyan/diploma/experiments/ppo/ft_ppo_bengali_mistral/5",
                   "/home/sslashinin/kovakimyan/diploma/experiments/ft_ppo_roberta_mistral/5",
                   "/home/sslashinin/kovakimyan/diploma/experiments/ppo/ft_ppo_hatespeech_detection_mistral/0",
                   "/home/sslashinin/kovakimyan/diploma/experiments/ppo/ft_ppo_bengali_mistral/10",
                   "/home/sslashinin/kovakimyan/diploma/experiments/ft_ppo_roberta_falcon/25",
                   "/home/sslashinin/kovakimyan/diploma/experiments/ppo/ft_ppo_bengali_falcon/15",
                   "/home/sslashinin/kovakimyan/diploma/experiments/ppo/ft_ppo_bengali_falcon/10",
                   
                   "/home/sslashinin/kovakimyan/diploma/experiments/ft_ppo_roberta_mistral/10",
                   "/home/sslashinin/kovakimyan/diploma/experiments/peft_fine_tuned_with_detoxification_rewards_falcon"
                  ]

for models in models_to_check:
    try:
          results.append({models: calc(models)})
    except:
          print("An exception occurred", models)
    

In [ ]:
results

In [ ]:
import json
import os

from deepeval.benchmarks.mmlu.task import MMLUTask

from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.benchmarks import MMLU
from deepeval.benchmarks import BigBenchHard

class Phi3(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        print("\n\n prompt =", prompt, "\n\n")
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Phi3_mini"

from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "microsoft/Phi-3-mini-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Phi3 = Phi3(model=model, tokenizer=tokenizer)
benchmark = MMLU(
    tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY],
    n_shots=3
)

print(Phi3.generate("Write me a joke"))

benchmark.evaluate(model=Phi3)
print(benchmark.overall_score)
print("Task-specific Scores: ", benchmark.task_scores)
# file_path2 = "./"
# Socres = benchmark.overall_score
# tasks = [MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY]
# benchmark = MMLU(tasks=tasks)

# dir_name, file_name = os.path.split(file_path2)
# new_file_name = "_ev_Hallu_" + file_name
# store_path = os.path.join(dir_name, new_file_name)
# with open(store_path, 'w', encoding='utf-8') as f:
#         json.dump(Socres, f, indent=4)

# print("Results saved to ", store_path)

/home/sslashinin/.conda/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [02:27<00:00, 73.73s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




 prompt = Write me a joke 




You are not running the flash-attention implementation, expect numerical differences.


<s> Write me a joke about a <*insert any profession here*> and an <*insert any object here*>.<|end|>


 prompt = The following are multiple choice questions (with answers) about high school computer science.

Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer: C

Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of

Processing high_school_computer_science:   1%|          | 1/100 [00:22<37:24, 22.67s/it]

...



 prompt = The following are multiple choice questions (with answers) about astronomy.

Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer: C

Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system

Processing astronomy:  91%|█████████ | 91/100 [43:03<02:56, 19.64s/it]



 prompt = The following are multiple choice questions (with answers) about astronomy.

Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer: C

Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system

Processing astronomy:  92%|█████████▏| 92/100 [44:08<04:26, 33.26s/it]



 prompt = The following are multiple choice questions (with answers) about astronomy.

Which of the following is an example of the use of a device on the Internet of Things (IoT) ?
A. A car alerts a driver that it is about to hit an object.
B. A hiker uses a G P S watch to keep track of her position.
C. A refrigerator orders milk from an online delivery service when the milk in the refrigerator is almost gone.
D. A runner uses a watch with optical sensors to monitor his heart rate.
Answer: C

Many Web browsers allow users to open anonymous windows. During a browsing session in an anonymous window, the browser does not record a browsing history or a list of downloaded files. When the anonymous window is exited, cookies created during the session are deleted. Which of the following statements about browsing sessions in an anonymous window is true?
A. The activities of a user browsing in an anonymous window will not be visible to people who monitor the user's network, such as the system

In [1]:
import json
import os

from deepeval.benchmarks.mmlu.task import MMLUTask

from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.benchmarks import MMLU
from deepeval.benchmarks import BigBenchHard

class Phi3(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        print("\n\n prompt =", prompt, "\n\n")
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Phi3_mini"

from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "tiiuae/falcon-7b-instruct"
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Phi3 = Phi3(model=model, tokenizer=tokenizer)
benchmark = MMLU(
    tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY],
    n_shots=3
)

/home/sslashinin/.conda/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 2/2 [00:41<00:00, 20.51s/it]


In [2]:
question = """The following are multiple choice questions (with answers) about high school computer science.

In python 3, which of the following is floor division?
A. /
B. //
C. %
D. |
Answer:Give only correct output 'A', 'B', 'C', or 'D'"""
print(Phi3.generate(question))



 prompt = The following are multiple choice questions (with answers) about high school computer science.

In python 3, which of the following is floor division?
A. /
B. //
C. %
D. |
Answer:Give only correct output 'A', 'B', 'C', or 'D' 




Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The following are multiple choice questions (with answers) about high school computer science.

In python 3, which of the following is floor division?
A. /
B. //
C. %
D. |
Answer:Give only correct output 'A', 'B', 'C', or 'D'

2.1 In which loop can you get the length of a string in Python?
A. For
B. While
C. For-Each
D. In a Loop
Answer:C. For-Each

In the programming language Java, which character is the wildcard character?
A. A
B. B
C. C
D. D
Answer:C. C

3. What is the { } syntax used for in


In [ ]:
benchmark.evaluate(model=Phi3)
print(benchmark.overall_score)
print("Task-specific Scores: ", benchmark.task_scores)

In [ ]:
import argparse
import os
import torch
import numpy as np
import pandas as pd
from categories import subcategories, categories
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import time

choices = ["A", "B", "C", "D"]


def format_subject(subject):
    l = subject.split("_")
    s = ""
    for entry in l:
        s += " " + entry
    return s


def format_example(df, idx, include_answer=True):
    prompt = df.iloc[idx, 0]
    k = df.shape[1] - 2
    for j in range(k):
        prompt += "\n{}. {}".format(choices[j], df.iloc[idx, j + 1])
    prompt += "\nAnswer:"
    if include_answer:
        prompt += " {}\n\n".format(df.iloc[idx, k + 1])
    return prompt


def gen_prompt(train_df, subject, k=-1):
    prompt = "The following are multiple choice questions (with answers) about {}.\n\n".format(
        format_subject(subject)
    )
    if k == -1:
        k = train_df.shape[0]
    for i in range(k):
        prompt += format_example(train_df, i)
    return prompt


@torch.no_grad()
def eval(args, subject, model, tokenizer, dev_df, test_df):
    cors = []
    all_probs = []
    answers = choices[: test_df.shape[1] - 2]

    for i in range(test_df.shape[0]):
        # get prompt and make sure it fits
        k = args.ntrain
        prompt_end = format_example(test_df, i, include_answer=False)
        train_prompt = gen_prompt(dev_df, subject, k)
        prompt = train_prompt + prompt_end

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

        while input_ids.shape[-1] > 2048:
            k -= 1
            train_prompt = gen_prompt(dev_df, subject, k)
            prompt = train_prompt + prompt_end
            input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

        label = test_df.iloc[i, test_df.shape[1] - 1]

        decoder_input_ids = tokenizer("", return_tensors="pt").input_ids.cuda()
        decoder_input_ids = model._shift_right(decoder_input_ids)
        logits = model(
            input_ids=input_ids, decoder_input_ids=decoder_input_ids
        ).logits.flatten()

        probs = (
            torch.nn.functional.softmax(
                torch.tensor(
                    [
                        logits[tokenizer("A").input_ids[0]],
                        logits[tokenizer("B").input_ids[0]],
                        logits[tokenizer("C").input_ids[0]],
                        logits[tokenizer("D").input_ids[0]],
                    ]
                ),
                dim=0,
            )
            .detach()
            .cpu()
            .numpy()
        )
        pred = {0: "A", 1: "B", 2: "C", 3: "D"}[np.argmax(probs)]

        cor = pred == label
        cors.append(cor)
        all_probs.append(probs)

    acc = np.mean(cors)
    cors = np.array(cors)

    all_probs = np.array(all_probs)
    print("Average accuracy {:.3f} - {}".format(acc, subject))

    return cors, acc, all_probs


def main(args):

    model_path = "microsoft/Phi-3-mini-128k-instruct"
    model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    device_map = "auto"

    model.eval()
    subjects = sorted(
        [
            f.split("_test.csv")[0]
            for f in os.listdir(os.path.join(args.data_dir, "test"))
            if "_test.csv" in f
        ]
    )

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    if not os.path.exists(os.path.join(args.save_dir, "results_{}".format(args.model))):
        os.makedirs(os.path.join(args.save_dir, "results_{}".format(args.model)))

    all_cors = []
    subcat_cors = {
        subcat: [] for subcat_lists in subcategories.values() for subcat in subcat_lists
    }
    cat_cors = {cat: [] for cat in categories}

    for subject in subjects:
        dev_df = pd.read_csv(
            os.path.join(args.data_dir, "dev", subject + "_dev.csv"), header=None
        )[: args.ntrain]
        test_df = pd.read_csv(
            os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
        )

        cors, acc, probs = eval(args, subject, model, tokenizer, dev_df, test_df)
        subcats = subcategories[subject]
        for subcat in subcats:
            subcat_cors[subcat].append(cors)
            for key in categories.keys():
                if subcat in categories[key]:
                    cat_cors[key].append(cors)
        all_cors.append(cors)

        test_df["{}_correct".format(args.model)] = cors
        for j in range(probs.shape[1]):
            choice = choices[j]
            test_df["{}_choice{}_probs".format(args.model, choice)] = probs[:, j]
        test_df.to_csv(
            os.path.join(
                args.save_dir, "results_{}".format(args.model), "{}.csv".format(subject)
            ),
            index=None,
        )

    for subcat in subcat_cors:
        subcat_acc = np.mean(np.concatenate(subcat_cors[subcat]))
        print("Average accuracy {:.3f} - {}".format(subcat_acc, subcat))

    for cat in cat_cors:
        cat_acc = np.mean(np.concatenate(cat_cors[cat]))
        print("Average accuracy {:.3f} - {}".format(cat_acc, cat))
    weighted_acc = np.mean(np.concatenate(all_cors))
    print("Average accuracy: {:.3f}".format(weighted_acc))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--ntrain", "-k", type=int, default=5)
    parser.add_argument("--ngpu", "-g", type=int, default=2)
    parser.add_argument("--data_dir", "-d", type=str, default="data")
    parser.add_argument("--save_dir", "-s", type=str, default="results")
    parser.add_argument(
        "--model",
        "-m",
        type=str,
        default="google/flan-t5-small",
    )
    args = parser.parse_args()
    main(args)